<a href="https://colab.research.google.com/github/alessandrotofani/Tesi_magistrale/blob/master/6_Federated_sistemato_low.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Overview: https://www.tensorflow.org/federated

Image classification tutorial: https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification

# Installation

In [1]:
!pip install --quiet tensorflow==2.3.0
!pip install --quiet tensorflow_federated==0.17.0
!pip install --quiet --upgrade nest_asyncio
# !pip install -q tfds-nightly

In [2]:
import nest_asyncio
nest_asyncio.apply()
%load_ext tensorboard

In [3]:
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd 
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sys 
sys.path.append('/content/drive/MyDrive/Tesi_magistrale/Tesi_magistrale')
import mf

In [5]:
data = pd.read_csv('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/data.csv')
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [6]:
data = mf.feature_engineering(data)
# data = mf.feature_scaling(data)
data = pd.get_dummies(data)

In [7]:
data = data[200000:300000]

In [8]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2)

In [9]:
def to_tensor(data, n_clients = 5):
  shuffled = data.sample(frac=1)
  result = np.array_split(shuffled, n_clients)  

  res = []

  new_res = []
  label = []

  for dataset in result:
    res.append(mf.feature_scaling(dataset))
  
  for subset in res:
    label.append(subset['isFraud'])
    new_res.append(subset.drop(columns = ['isFraud']).to_numpy())

  dataset = {}
  for i in range(n_clients):
    dataset[i] = tf.data.Dataset.from_tensor_slices((new_res[i], label[i]))
  return dataset

In [10]:
dataset = to_tensor(train_data)
test_set = to_tensor(test_data)

In [11]:
NUM_CLIENTS = 5
NUM_EPOCHS = 10
BATCH_SIZE = 1000
SHUFFLE_BUFFER = 10
PREFETCH_BUFFER = 10

def preprocess(dataset):

  def batch_format_fn(e1, e2):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    return collections.OrderedDict(
        x = tf.cast(e1, tf.float32),
        y = tf.cast(e2, tf.int32))
      #  x=tf.reshape(element, [-1, 1103]),
      #  y=tf.reshape(target, [-1, 1]))
        # element)

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

preprocessed_example_dataset = preprocess(dataset[0])

# sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
#                                      next(iter(preprocessed_example_dataset)))

# sample_batch

In [12]:
def make_federated_data(dataset):
  federated = []
  for i in dataset:
    federated.append(preprocess(dataset[i]))
  return federated

federated_train_data = make_federated_data(dataset)

print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))

Number of client datasets: 5
First dataset: <PrefetchDataset shapes: OrderedDict([(x, (None, 1103)), (y, (None,))]), types: OrderedDict([(x, tf.float32), (y, tf.int32)])>


In [13]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input

def create_keras_model():
  model = Sequential()
  model.add(Input(shape=(1103,))) 
  # model.add(Dense(1024, activation='relu')) 
  # model.add(Dropout(0.2)) 
  # model.add(Dense(512, activation='relu')) 
  # model.add(Dropout(0.2)) 
  model.add(Dense(256, activation='relu')) 
  model.add(Dropout(0.2))
  model.add(Dense(128, activation='relu')) 
  model.add(Dropout(0.2))
  model.add(Dense(24, activation='relu')) 
  model.add(Dropout(0.2))
  model.add(Dense(4, activation='relu')) 
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))
  return model

def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  soglia = 0.2
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=[tf.keras.metrics.BinaryAccuracy(), 
               tf.keras.metrics.Recall(thresholds=soglia),
               tf.keras.metrics.Precision(thresholds=soglia)])
  
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,  
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.2), #0.4
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.6)) #0.8

state = iterative_process.initialize()

state, metrics = iterative_process.next(state, federated_train_data)
print('round  1, metrics={}'.format(metrics))

round  1, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('binary_accuracy', 0.9580236), ('recall', 0.14410147), ('precision', 0.08394908), ('loss', 0.18418264)]))])


In [14]:
NUM_ROUNDS = 141
for round_num in range(2, NUM_ROUNDS):
  state, metrics = iterative_process.next(state, federated_train_data)
  print('round {:2d}, metrics={}'.format(round_num, metrics))

round  2, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('binary_accuracy', 0.9590999), ('recall', 0.14425428), ('precision', 0.12817727), ('loss', 0.16821112)]))])
round  3, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('binary_accuracy', 0.9590999), ('recall', 0.13401589), ('precision', 0.16098242), ('loss', 0.1620772)]))])
round  4, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('binary_accuracy', 0.9590999), ('recall', 0.11742054), ('precision', 0.18100443), ('loss', 0.15834728)]))])
round  5, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('binary_accuracy', 0.9590998), ('

In [15]:
evaluation = tff.learning.build_federated_evaluation(model_fn)
train_metrics = evaluation(state.model, federated_train_data)

In [16]:
federated_test_data = make_federated_data(test_set)

In [17]:
test_metrics = evaluation(state.model, federated_test_data)
str(test_metrics)

"OrderedDict([('binary_accuracy', 0.9675001), ('recall', 0.38231632), ('precision', 0.6868009), ('loss', 0.11223874)])"

In [ ]:
logdir = "/tmp/logs/scalars/training/"
summary_writer = tf.summary.create_file_writer(logdir)
state = iterative_process.initialize()
with summary_writer.as_default():
  for round_num in range(1, NUM_ROUNDS):
    state, metrics = iterative_process.next(state, federated_train_data)
    for name, value in metrics['train'].items():
      tf.summary.scalar(name, value, step=round_num)

In [ ]:
!ls {logdir}
%tensorboard --logdir {logdir} --port=0